<div align="center">
  <img src="https://www.dropbox.com/s/vold2f3fm57qp7g/ECE4179_5179_6179_banner.png?dl=1" alt="ECE4179/5179/6179 Banner" style="max-width: 60%;"/>
</div>

<div align="center">


# PyTorch Optimizers

</div>


Welcome to Week 5 of ECE4179/5179/6179! This week, we continue our study of optimization techniques, focusing on those provided by PyTorch. Specifically, we will learn about widely-used methods like Stochastic Gradient Descent (SGD)[1], SGD with momentum, and Adam[2]. These optimizers are essential tools for training deep learning models and are foundational in modern machine learning. Let’s get started and see how these optimizers perform in practice!

[1] Bottou, L. (2010). *Large-Scale Machine Learning with Stochastic Gradient Descent*. Proceedings of COMPSTAT, 177-186.  
[2] Kingma, D. P., & Ba, J. (2015). *Adam: A Method for Stochastic Optimization*. ICLR'14.



In [1]:
# as always, we start by importing the necessary packages

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm


import torch
from torch.optim import Adam, SGD


# Set random seed
RND_SEED = 42
np.random.seed(RND_SEED)
torch.manual_seed(RND_SEED)

We will use the following helper functions for visualization purposes in this notebook. These functions will allow us to plot the optimization paths and gain insights into how different optimizers behave during training.

In [2]:
def plot_camel_back_history(hist_x):
    """
    Plot the camelback function and the history of gradient descent.
    """

    # Define the camelback function using a lambda function
    f_x = lambda x: 4 * x**2 - 2.1 * x**4 + (1 / 3) * x**6 + x * torch.cos(x)

    # Convert the history points to a tensor
    hist_x = torch.tensor(hist_x, dtype=torch.float32)
    plt.figure(figsize=(10, 5))

    # Calculate the function values for each point in the history
    hist_f = f_x(hist_x)

    # Plot the function first to ensure it is in the background
    x = torch.linspace(-2, 2, 100)
    plt.plot(x, f_x(x), color="blue", label="f(x)", zorder=1)

    # Plot arrows, text labels, and circles to visualize the gradient descent path
    for i in range(len(hist_x) - 1):
        # Calculate the difference between consecutive points
        dx = hist_x[i + 1] - hist_x[i]
        dy = hist_f[i + 1] - hist_f[i]

        # Annotate the path with arrows showing the direction of descent
        plt.annotate(
            "",
            xy=(hist_x[i + 1], hist_f[i + 1]),
            xytext=(hist_x[i], hist_f[i]),
            arrowprops=dict(
                facecolor="#FF5F1F",
                edgecolor="#EC5800",
                shrink=0,
                width=2,
                headwidth=8,
                headlength=10,
            ),
            zorder=4,
        )

    # Scatter plot to mark the points along the descent path
    plt.scatter(hist_x, hist_f, c="k", s=50, zorder=5)  # Adding a circle at each point

    # Label the axes
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.legend()
    plt.show()


# ===================================


def plot_ackley(hist_x=None):
    """
    Plot the Ackley function and optionally the trajectory of an optimizer.

    Parameters:
    - hist_x: Optional. A list of [x, y] coordinates representing the optimization trajectory.
    """

    # Define the Ackley function using a lambda function
    ackley_func = (
        lambda v: -20
        * torch.exp(-0.2 * torch.sqrt(0.5 * (v[:, 0] ** 2 + v[:, 1] ** 2)))
        - torch.exp(
            0.5
            * (torch.cos(2 * torch.pi * v[:, 0]) + torch.cos(2 * torch.pi * v[:, 1]))
        )
        + torch.exp(torch.tensor(1.0))
        + 20
    )

    # Grid setup
    num_pnts = 100
    x = torch.linspace(-5, 5, num_pnts)
    y = torch.linspace(-5, 5, num_pnts)
    X_grid, Y_grid = torch.meshgrid(x, y, indexing="ij")

    v = torch.stack([X_grid.reshape(-1), Y_grid.reshape(-1)], dim=1)
    Z = ackley_func(v).reshape(num_pnts, num_pnts)

    # Plot the contour plot with the trajectory
    fig, ax = plt.subplots(figsize=(8, 8))
    contour_plot = ax.contourf(X_grid, Y_grid, Z, levels=50, cmap=cm.coolwarm)

    if hist_x is not None:
        # Plot the trajectory
        x_history = [coord[0] for coord in hist_x]
        y_history = [coord[1] for coord in hist_x]
        for i in range(1, len(x_history)):
            ax.annotate(
                "",
                xy=(x_history[i], y_history[i]),
                xytext=(x_history[i - 1], y_history[i - 1]),
                arrowprops=dict(
                    facecolor="black", edgecolor="black", arrowstyle="->", lw=2
                ),
            )

        # Add a green star at the final point
        ax.plot(x_history[-1], y_history[-1], "g*", markersize=15, label="Final Point")
        ax.legend()

    plt.colorbar(contour_plot, ax=ax, shrink=0.75)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_title("Contour Plot of Ackley Function with Trajectory")
    ax.set_aspect("equal")

    plt.show()

### Automatic Differentiation with PyTorch

We start understanding PyTorch optimizers by learning about Automatic Differentiation (AD).

Consider the function:

\begin{align}
f(x) = x^3 - 3x^2 + 2x\;. 
\end{align}


What is the derivative of this function at $x=2$? 

In PyTorch, you can compute the derivative of a variable (or more generally, a computational graph) by following these steps:

1. Set the attribute `requires_grad=True` for the variable $x$. This tells PyTorch to track operations on this variable for gradient computation.
2. Call the `.backward()` method on the computed value of $f(x)$. This triggers PyTorch's automatic differentiation to compute the gradient of $f(x)$ with respect to $x$.




<div style="background-color: #3352FF; color: white; padding: 10px; border-radius: 5px;">

### <span style="color: pink;">Task #1.</span>  AD basics
Use PyTorch to obtain the derivative of $f(x) = x^3 - 3x^2 + 2x$ at $x=2$. 

</div>



In [5]:
# Define the input tensor with requires_grad=True
x = torch.tensor(2.0, requires_grad=True)

# Define the function
f_x = torch.pow(x, 3) - 3 * torch.pow(x, 2) + 2 * x

# Compute the derivative using backward()
f_x.backward()

# Display the gradient
print(f"Gradient of f(x) at x = {x.item()} is: {x.grad.item()}")

Gradient of f(x) at x = 2.0 is: 2.0


Consider $\mathbf{x} \in \mathbb{R}^2$ and $\mathbf{w} \in \mathbb{R}^2$. Let:

\begin{align*}
    p = \sigma(\mathbf{w}^\top \mathbf{x})\;,
\end{align*}

where $\sigma(z) = \frac{1}{1 + \exp(-z)}$ is the sigmoid function. The entropy of $p \in [0, 1]$, which is a measure of uncertainty, is defined as:

\begin{align*}
    f(p) = -p \log(p)
\end{align*}


<div style="background-color: #3352FF; color: white; padding: 10px; border-radius: 5px;">

### <span style="color: pink;">Task #2.</span>  Gradient of the Entropy Function


Your task is to obtain $\nabla_{\mathbf{x}} f$ and $\nabla_{\mathbf{w}} f$ at 
\begin{align*}
    \mathbf{x} = \begin{pmatrix} 1.0 \\ 2.0 \end{pmatrix} \quad \text{and} \quad 
    \mathbf{w} = \begin{pmatrix} 0.5 \\ -1.5 \end{pmatrix}\;.
\end{align*}

Since you already know the BP algorithm, you should be able to solve this task by hand. So, work it out with your tablemates and complete the code below to obtain the solution.
</div>


<details>
<summary style="color: yellow; font-weight: bold;">Hint</summary>

Use the chain rule (see the example below):

\begin{align*}
\nabla_{\mathbf{x}} f = \frac{\partial f}{\partial p} \frac{\partial p}{\partial z} \frac{\partial z}{\partial \mathbf{x}} 
\end{align*}

Also, note the following partial derivatives:

\begin{align*}
\frac{\partial }{\partial z}\sigma(z) = \sigma(z)\big(1 - \sigma(z)\big)\;,
\end{align*}

\begin{align*}
\frac{\partial }{\partial \mathbf{x}} \mathbf{w}^\top \mathbf{x} = \mathbf{w}\;,
\end{align*}

\begin{align*}
\frac{\partial }{\partial \mathbf{w}} \mathbf{w}^\top \mathbf{x} = \mathbf{x}\;.
\end{align*}

</details>

In [7]:
# Forward pass
x = torch.tensor([1.0, 2.0])
w = torch.tensor([0.5, -1.5])

# Compute the linear term (check the dot product function in the PyTorch documentation)
z = torch.dot(w, x)

# Apply the sigmoid function to get p (check the sigmoid function in the PyTorch documentation)
p = torch.sigmoid(z)

# Compute the entropy function
f = -p * torch.log(p)

# Backward pass
dz_dw = x
dz_dx = w
dp_dz = p * (1 - p)
df_dp = -torch.log(p) - 1

df_dx = df_dp * dp_dz * dz_dx
df_dw = df_dp * dp_dz * dz_dw

print(f"df_dx = {df_dx}")
print(f"df_dw = {df_dw}")

df_dx = tensor([ 0.0553, -0.1660])
df_dw = tensor([0.1107, 0.2214])


<div style="background-color: #3352FF; color: white; padding: 10px; border-radius: 5px;">

### <span style="color: pink;">Task #3.</span>  Gradient of the Entropy Function using AD

Repeat the previous task but this time with PyTorch's automatic differentiation.

</div>

In [ ]:
# Define the inputs
x = torch.tensor([1.0, 2.0], requires_grad=True)
w = torch.tensor([0.5, -1.5], requires_grad=True)

# Compute the linear term (check the dot product function in the PyTorch documentation)
z = torch.dot(w, x)

# Apply the sigmoid function to get p (check the sigmoid function in the PyTorch documentation)
p = torch.sigmoid(z)

# Compute the entropy function
f = -p * torch.log(p)

# Compute the gradients using the backward() function
f.backward()

# Print the gradients
print(f"Gradient with respect to x: {x.grad}")
print(f"Gradient with respect to w: {w.grad}")

### PyTorch Optimizers

In PyTorch, the `torch.optim` module implements several optimizers. The general workflow for using an optimizer is as follows:

1. Define what to optimize (i.e., parameters). To do this, set the `requires_grad=True` attribute for the PyTorch tensor(s) that you want to optimize.
2. Define the optimizer by passing the parameters to be optimized and the hyperparameters of the optimizer (e.g., learning rate).
3. Compute the objective of the optimization problem (e.g., the loss function).
4. Compute the gradients of the objective with respect to the parameters. This is done by calling the `.backward()` method on the computed value of the objective.
5. Update the parameters using the `.step()` method of the optimizer.

The simplest optimizer is SGD (Stochastic Gradient Descent), which is the stochastic version of the Gradient Descent algorithm. The following code snippet shows how to use the SGD optimizer to minimize the function \( f(x) = x^2 \):

```python
import torch
import torch.optim as optim

# Define the variable to optimize
x = torch.tensor([1.0], requires_grad=True)

# Define the optimizer
optimizer = optim.SGD([x], lr=0.1, momentum=0.0)

# Compute the objective
f_x = x**2

# Compute the gradients
f_x.backward()

# Update the variable
optimizer.step()
```

In this example, the optimizer performs one step of the SGD algorithm to minimize the function $f(x) = x^2$ using a learning rate of 0.1.

### The Six-hump Camelback function

Let's practice by optimizing our beloved six-hump camelback function defined as:
\begin{align}
    f(x) = 4x^2 - 2.1x^4 + \frac{1}{3}x^6 + x\cos(x)
\end{align}


<div align="center">
  <img src="https://www.nicepng.com/png/detail/245-2450208_camels-clipart-2-hump-cartoon-camel-two-humps.png" alt="cute camel" style="max-width: 40%;"/>
</div>


<div style="background-color: #3352FF; color: white; padding: 10px; border-radius: 5px;">

### <span style="color: pink;">Task #4.</span>  Minimizing the Six-hump Camelback function with SGD

SGD optimizer is the stochastic version of the Gradient Descent algorithm. Use the SGD optimizer to minimize the six-hump camelback function. For this task, run the optimizer for 10 iterations, and start from $x=2.0$. Make sure the momentum is set to 0.0.

</div>


In [ ]:
num_iterations = 10 # Keep this as 10 but feel free to change it to see the behaviour
x = torch.tensor(2.0, requires_grad=True)

# Define the optimizer
optimizer = SGD([x], 
                lr=, # <--- YOUR CODE HERE 
                momentum=0.0) # Keep this as 0.0 in this task

def camel_back_func(x):
    f_x = # <--- YOUR CODE HERE
    return f_x

hist_x = []
for iter in range(num_iterations):
    hist_x.append(x.clone().detach())

    # Clear the gradient (this is needed in PyTorch)
    optimizer.zero_grad()

    # Compute the function
    f_x = # <--- YOUR CODE HERE 

    # Compute the gradient
    f_x # <--- YOUR CODE HERE 

    # Update the variable
    optimizer.step()

    

    print(f"Iteration {iter+1 :2}: x = {x.item():6.3f}, f(x) = {f_x.item():6.3f}")


plot_camel_back_history(hist_x)

To assist you with the rest of experiments in this notebook, consider using the following helper function: `run_optimizer`. This function allows you to easily experiment with different optimization strategies and track the progress over multiple steps. The function allows you to optimize a given function starting from an initial point and using your choice of optimizer.

```python
def run_optimizer(func, x_init, optimizer_class, optimizer_params, n_steps):
    ...
```

**Parameters:**
- `func`: The function to optimize. This should be a PyTorch function that takes a 2D input (e.g., a single sample with 2 features) and returns a scalar output.
- `x_init`: The initial value of $\mathbf{x}$.
- `optimizer_class`: The optimizer class you want to use (e.g., `torch.optim.SGD`, `torch.optim.Adam`).
- `optimizer_params`: A dictionary of parameters for the optimizer (e.g., `{'lr': 0.01, 'momentum': 0.9}`).
- `n_steps`: The number of optimization steps to run.

**Returns:**
- `hist_x`: A list containing the history of $\mathbf{x}$ values over the optimization steps.
- `hist_grad`: A list containing the history of gradient values over the optimization steps.




In [ ]:
def run_optimizer(func, x_init, optimizer_class, optimizer_params, n_steps):
    """
    Run the optimizer on a given function starting from an initial point.

    Parameters:
    - func: The function to optimize
    - x_init: Initial value of x (starting point), should be a list or array-like with two elements for bivariate functions
    - optimizer_class: The class of the optimizer (e.g., torch.optim.SGD)
    - optimizer_params: A dictionary of parameters for the optimizer (e.g., {'lr': 0.01, 'momentum': 0.9})
    - n_steps: Number of optimization steps

    Returns:
    - hist_x: History of x values over the optimization steps
    - hist_grad: History of gradient values over the optimization steps
    """

    # Ensure x_init is a tensor and keep it consistent
    x = torch.tensor(x_init, requires_grad=True, dtype=torch.float32)
    hist_x = [x.clone().detach().numpy()]  # Store the initial point as a numpy array
    hist_grad = []

    # Initialize the optimizer with the given parameters
    optimizer = optimizer_class([x], **optimizer_params)

    for i in range(n_steps):
        optimizer.zero_grad()  # Clear previous gradients

        # Pass x directly to the function
        f_x = func(x)

        f_x.backward()  # Compute the gradients
        optimizer.step()  # Update x using the optimizer

        # Record the history of x and gradients
        hist_x.append(
            x.clone().detach().numpy()
        )  # Append the new x value as a numpy array
        hist_grad.append(x.grad.clone().detach().numpy())

    return hist_x, hist_grad

### From 1D to 2D

Recall that the Ackley function is a test problem for optimization algorithms. It is defined as:

\begin{equation}
    f(x,y) = -20 \exp \left( -0.2 \sqrt{0.5(x^2 + y^2)} \right) - \exp \left( 0.5 \left( \cos(2\pi x) + \cos(2\pi y) \right) \right) + e + 20
\end{equation}

The Ackley function has a global minimum at $(0,0)$ with $f(0,0) = 0$.

<div style="background-color: #3352FF; color: white; padding: 10px; border-radius: 5px;">

### <span style="color: pink;">Task #5.</span>  Minimizing the Ackley function with SGD

Use the SGD optimizer, and start from point $x_0 = (4, 3)$ to find the minimum of the Ackley function. Pick learning rate yourself and run the algorithm for 25 iterations.

</div>


In [ ]:
def ackley_func(v):
    if type(v) == np.ndarray:
        v = torch.tensor(v, dtype=torch.float32)

    pi = torch.pi
    x = v[0]  # Extract the x coordinates
    y = v[1]  # Extract the y coordinates
    x_sq = x**2
    y_sq = y**2

    # Calculate the exponential term
    exp_term = torch.exp(-0.2 * torch.sqrt(0.5 * (x_sq + y_sq)))
    # Calculate the cosine term
    cos_term = torch.cos(2 * pi * x) + torch.cos(2 * pi * y)

    # Compute the Ackley function value
    f_xy = (
        -20 * exp_term - torch.exp(0.5 * cos_term) + torch.exp(torch.tensor(1.0)) + 20
    )
    return f_xy


plot_ackley()

In [ ]:
num_steps = 25
x_init = # <--- YOUR CODE HERE 



optimizer4 = SGD
optimizer_params4 = {'lr': , # <--- YOUR CODE HERE 
                     'momentum': 0.0}

hist_x4, hist_grad4 = run_optimizer(# <--- YOUR CODE HERE 

plot_ackley(hist_x4)

# Momentum

An extension of the SGD algorithm is the SGD with momentum. The momentum term helps accelerate the optimization process by accumulating the gradients over time. The update rule for the SGD with momentum is given by:

\begin{align}
    m_{t+1} &= \beta m_t + \nabla f(x_t) \\
    x_{t+1} &= x_t - \eta m_{t+1}\;,
\end{align}
where $m_t$ is the momentum term, $\beta$ is the momentum coefficient, and $\eta$ is the learning rate. 


<div style="background-color: #3352FF; color: white; padding: 10px; border-radius: 5px;">
### <span style="color: pink;">Task #6.</span>  Minimizing the Ackley function with SGD-Momentum

Use the SGD optimizer, and start from point $x_0 = (4, 3)$ to find the minimum of the Ackley function. Pick learning rate, and the momentum coefficient yourself and run the algorithm for 25 iterations. Compare the results with the previous task.

</div>

In [ ]:
num_steps = 25
x_init = [4.0,3.0]


optimizer5 = SGD
optimizer_params5 = {'lr': # <--- YOUR CODE HERE 
                     'momentum': # <--- YOUR CODE HERE 
                     }

hist_x5, hist_grad5 = run_optimizer( # <--- YOUR CODE HERE 

plot_ackley(hist_x5)

# Adaptive Moment Estimation (Adam)

An optimizer widely used in practice is the Adam optimizer. One difficulty of the SGD is adjusting its learning rate. Ideally, we would like to have a large learning rate in smooth regions (low curvature) where the gradient is small, and hence the progress is slow. Conversely, we would like to have a small learning rate in regions with high curvature, where the gradient is large. The Adam optimizer adapts the learning rate for each parameter by using the first and second moments of the gradients. The update rule (simplified for clarity) for the Adam optimizer is given by:

\begin{align}
    m_{t+1} &= \beta_1 m_t + (1 - \beta_1) \nabla f(x_t) \\
    v_{t+1} &= \beta_2 v_t + (1 - \beta_2) \nabla f(x_t)^2 \\
    x_{t+1} &= x_t - \eta \frac{{m}_{t+1}}{\sqrt{{v}_{t+1}} + \epsilon}\;,
\end{align}
where $m_t$ and $v_t$ are the first and second moments of the gradients, respectively, $\beta_1$ and $\beta_2$ are the momentum coefficients, $\eta$ is the learning rate, and $\epsilon$ is a small constant to prevent division by zero. 

<div style="background-color: #3352FF; color: white; padding: 10px; border-radius: 5px;">
### <span style="color: pink;">Task #7.</span>  Minimizing the Ackley function with Adam

Use the Adam optimizer, and start from point $x_0 = (4, 3)$ to find the minimum of the Ackley function. Pick learning rate and run the algorithm for 25 iterations. Compare the results with the previous tasks.

</div>

In [ ]:
num_steps = 25
x_init = [4.0,3.0]


optimizer7 = Adam
optimizer_params7 = {'lr': # <--- YOUR CODE HERE 
                     }

hist_x7, hist_grad7 = run_optimizer # <--- YOUR CODE HERE 

plot_ackley(hist_x7)

### Conclusion

In this notebook, we covered the key concepts related to optimization in PyTorch:

- We began by understanding automatic differentiation and explored how PyTorch computes gradients.
- We manually computed gradients for a function involving the sigmoid and entropy to better understand the backpropagation algorithm.
- Finally, we used PyTorch’s built-in optimizers like SGD to optimize functions and analyze the impact of different hyperparameters.

